<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prelude" data-toc-modified-id="Prelude-0"><span class="toc-item-num">0&nbsp;&nbsp;</span>Prelude</a></span><ul class="toc-item"><li><span><a href="#Rename-your-notebook" data-toc-modified-id="Rename-your-notebook-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Rename your notebook</a></span></li></ul></li><li><span><a href="#Latent-Semantic-Indexing-(LSI)" data-toc-modified-id="Latent-Semantic-Indexing-(LSI)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Latent Semantic Indexing (LSI)</a></span><ul class="toc-item"><li><span><a href="#Build-the-vocabulary-by-selecting-top-k-frequent-words" data-toc-modified-id="Build-the-vocabulary-by-selecting-top-k-frequent-words-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Build the vocabulary by selecting top-k frequent words</a></span></li><li><span><a href="#Construct-the-term-document-matrix" data-toc-modified-id="Construct-the-term-document-matrix-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Construct the term document matrix</a></span></li><li><span><a href="#Perform-LSI-by-selecting-the-first-100-largest-singular-values-of-the-term-document-matrix" data-toc-modified-id="Perform-LSI-by-selecting-the-first-100-largest-singular-values-of-the-term-document-matrix-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Perform LSI by selecting the first 100 largest singular values of the term document matrix</a></span></li><li><span><a href="#Transform-the-given-query" data-toc-modified-id="Transform-the-given-query-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Transform the given query</a></span><ul class="toc-item"><li><span><a href="#Treat-the-query-as-another-document" data-toc-modified-id="Treat-the-query-as-another-document-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Treat the query as another document</a></span></li><li><span><a href="#Compute-the-query-vector-from-its-document-vector" data-toc-modified-id="Compute-the-query-vector-from-its-document-vector-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Compute the query vector from its document vector</a></span></li></ul></li><li><span><a href="#Retrieve-top-10-relevant-documents" data-toc-modified-id="Retrieve-top-10-relevant-documents-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Retrieve top-10 relevant documents</a></span><ul class="toc-item"><li><span><a href="#Fill-in-the-correct-parameters-in-the-following-function" data-toc-modified-id="Fill-in-the-correct-parameters-in-the-following-function-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Fill in the correct parameters in the following function</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#Compute-F1-score-at-10-between-the-oracle-and-your-result" data-toc-modified-id="Compute-F1-score-at-10-between-the-oracle-and-your-result-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Compute F1-score at 10 between the oracle and your result</a></span></li></ul></li><li><span><a href="#Visualization" data-toc-modified-id="Visualization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Visualization</a></span><ul class="toc-item"><li><span><a href="#Compute-the-term-vectors-using-two-principal-concepts" data-toc-modified-id="Compute-the-term-vectors-using-two-principal-concepts-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Compute the term vectors using two principal concepts</a></span></li><li><span><a href="#Explain-the-scatter-plot-of-the-term-vectors" data-toc-modified-id="Explain-the-scatter-plot-of-the-term-vectors-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Explain the scatter plot of the term vectors</a></span></li></ul></li></ul></div>

# Prelude

***Distributed Information Systems***

***Prof. Karl Aberer***

***Midterm Exam, Spring Semester 2019***

The following materials are allowed: exercise sheets and solutions, past exams with your own solution, personally written notes and personally collected documentation.

The exam will be held on your computer, but digital communication by any means is strictly prohibited. You are not allowed to use any form of Web browsing, except to connect to Moodle and to download and upload the exam. By participating to this exam you agree to these conditions.

These are the instructions for the exam:

1. You are not allowed to leave the examination room in the first 20 and the last 15 minutes of the exam.
* We will publish 15 minutes before the end of the exam a password for uploading your solutions on [Moodle](https://moodle.epfl.ch/course/view.php?id=4051).
* It is not recommended to leave the exam before the password is published. If you need to leave earlier, contact us.

## Rename your notebook
Replace SciperNo with your **personal SCIPER Number**.

In [1]:
#Required libraries test

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import math
from nltk.stem import PorterStemmer, WordNetLemmatizer
import os
import string
import numpy as np
from collections import Counter
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from collections import Counter
from operator import itemgetter
from pprint import pprint
import nltk
import matplotlib.pyplot as plt
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english')).union(set(stopwords.words('french')))

[nltk_data] Downloading package punkt to /Users/liu.jiaqi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liu.jiaqi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def read_corpus():
    '''Reads corpus from files.'''
    
    documents = []
    orig_docs = []
    DIR = './'
    tknzr = TweetTokenizer()
    with open("epfldocs.txt", encoding = "utf-8") as f:
        content = f.readlines()
    for text in content:
        orig_docs.append(text)
        # split into words
        tokens = tknzr.tokenize(text)
        # convert to lower case
        tokens = [w.lower() for w in tokens]
        # remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        words = [word for word in stripped if word.isalpha()]
        # filter out stop words
        words = [w for w in words if not w in stop_words]

        documents.append(' '.join(words))
    return documents, orig_docs

In [3]:
documents, orig_docs = read_corpus()

In [4]:
assert(len(documents) == 1075)

In [6]:
documents

['congrats ethen place epflen place',
 'thats standing ovation epflen',
 'time open farms pitch openfood opendata epfl',
 'good opportunity ethen epflen epflpark uzhnewsen',
 'exciting news world university rankings subject computer science eth epfl congrats httpstcoarslxzoshq',
 'epfl cryptotokenize free energy httpstcozyenraseir get rich',
 'scientists discover hidden patterns brain activity httpstcohblrytdmtt epfl',
 'edacyhq edacy invest',
 'deadline breast cancer research funding epfl againstbc',
 'robots transform furniture epfl november kamiahok',
 'microsoft president bradsmi epfl ebugnion digitalgenevaconvention digitaltrust important topics ethrat epflen',
 'atelier travail future traversée rade geneve cantongeneve epfl hslu etc hesso hei hepia heiafr',
 'formidable fan projet time machine donc possible revoir conférence chaîne youtube si jai bien lu les précédents commentaires merci signaler quand cette dernière ligne archives numerique',
 'members wef global university lead

In [7]:
orig_docs

['Congrats to @ETH_en (5th place) and @EPFL_en (10th place)! https://t.co/8RLp0zZhyb\n',
 "If that's not a standing ovation at @EPFL_en ... 🏆🙃😜👾🚀🎉 https://t.co/23kjIG8Pxt\n",
 "Time for Open Farm's pitch #openfood #Opendata #EPFL https://t.co/7lKKao5780\n",
 'Good opportunity for @ETH_en  @EPFL_en  @EPFL_Park @uzh_news_en https://t.co/v3tOjTYjQD\n',
 'Exciting News: "World University Rankings 2016-2017 by subject: computer science" No1 @ETH &amp; @EPFL on No8. Congrats https://t.co/ARSlXZoShQ\n',
 '@EPFL  crypto-tokenize FREE ENERGY https://t.co/vH9qgLoqrr https://t.co/6uTnJ68pSC https://t.co/GTXLhI6LTE https://t.co/ZYenRAsEIr GET RICH\n',
 'Scientists discover hidden patterns of brain activity https://t.co/HBLRYtdMtT #epfl\n',
 '@EdacyHQ  With #EDACY Invest in yourself https://t.co/X2Y212DKUP\n',
 'Deadline 19.05.2017 - AGAINST BREAST CANCER I RESEARCH FUNDING https://t.co/l4mhQA7upb #epfl @againstbc\n',
 'Robots #transform #into #furniture #at #EPFL #November #KamiAhok https://t.co/j

# Latent Semantic Indexing (LSI)

## Build the vocabulary by selecting top-k frequent words

In [9]:
def create_vocabulary_frequency(corpus, vocab_len):
    '''Select top-k (k = vocab_len) words in term of frequencies as vocabulary'''
    
    count = {} # dictionary that contains the frequency of each word count[word] = freq
    for document in corpus:
        for word in document.split():
            if word not in count.keys():
                count[word] = 1
            else:
                count[word] += 1

    sorted_count_by_freq = sorted(count.items(), key=lambda kv: kv[1], reverse=True)
    vocabulary = [x[0] for x in sorted_count_by_freq[:vocab_len+1]]
    
    return vocabulary



'''
# computing the search result (get the topk documents)
def search_vec(query, topk):
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(documents))]
    scores.sort(key=lambda x: -x[0])
    for i in range(topk):
            print(original_documents[scores[i][1]])


vocabulary = set([item for sublist in documents for item in sublist])
vocabulary = [word for word in vocabulary if word not in stopwords.words('english')]
vocabulary.sort()
'''

"\n# computing the search result (get the topk documents)\ndef search_vec(query, topk):\n    q = query.split()\n    q = [stemmer.stem(w) for w in q]\n    query_vector = vectorize(q, vocabulary, idf)\n    scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(documents))]\n    scores.sort(key=lambda x: -x[0])\n    for i in range(topk):\n            print(original_documents[scores[i][1]])\n\n\nvocabulary = set([item for sublist in documents for item in sublist])\nvocabulary = [word for word in vocabulary if word not in stopwords.words('english')]\nvocabulary.sort()\n"

In [10]:
vocab_freq = create_vocabulary_frequency(documents, 1000)

In [11]:
vocab_freq

['epfl',
 'epflen',
 'via',
 'les',
 'new',
 'lausanne',
 'switzerland',
 'vdtech',
 'research',
 'swiss',
 'science',
 'students',
 'innovation',
 'first',
 'martinvetterli',
 'day',
 'solar',
 'great',
 'open',
 'unil',
 'data',
 'technology',
 'learning',
 'thanks',
 'ethen',
 'work',
 'prof',
 'today',
 'startup',
 'talk',
 'see',
 'days',
 'next',
 'congrats',
 'one',
 'good',
 'suisse',
 'researchers',
 'campus',
 'center',
 'time',
 'eth',
 'energy',
 'robots',
 'join',
 'us',
 'epflcampus',
 'cc',
 'made',
 'lab',
 'dgtswitzerland',
 'world',
 'brain',
 'robot',
 'looking',
 'meeting',
 'people',
 'drone',
 'get',
 'future',
 'human',
 'mooc',
 'perovskite',
 'could',
 'epflenac',
 'free',
 'robotics',
 'event',
 'marcelsalathe',
 'conference',
 'proud',
 'light',
 'workshop',
 'forward',
 'week',
 'school',
 'der',
 'president',
 'universities',
 'design',
 'read',
 'im',
 'university',
 'scientists',
 'watch',
 'cours',
 'team',
 'article',
 'last',
 'top',
 'take',
 'excitin

## Construct the term document matrix
In this question, you need to construct the term document matrix given the vocabulary and the set of documents.
The value of a cell (i,j) is the term frequency of the term i in document j.
This is **different** from the definition from the lecture.

In [17]:
def construct_term_document_matrix(vocabulary, documents):
    matrix = np.zeros((len(vocabulary), len(documents)))
    for j, document in enumerate(documents):
        counter = Counter(document.split())
        for i, word in enumerate(vocabulary):
            if word not in counter.keys():
                matrix[j][i] = 0
            else:
                matrix[j][i] = counter[word]
            # YOUR CODE HERE
            
    return matrix


'''
# Function to generate the vector for a document (with normalisation)
def vectorize(document, vocabulary, idf):
    vector = [0]*len(vocabulary)
    counts = Counter(document)
    max_count = counts.most_common(1)[0][1]
    for i,term in enumerate(vocabulary):
        vector[i] = idf[term] * counts[term]/max_count
    return vector
'''

'\n# Function to generate the vector for a document (with normalisation)\ndef vectorize(document, vocabulary, idf):\n    vector = [0]*len(vocabulary)\n    counts = Counter(document)\n    max_count = counts.most_common(1)[0][1]\n    for i,term in enumerate(vocabulary):\n        vector[i] = idf[term] * counts[term]/max_count\n    return vector\n'

In [14]:
documents

['congrats ethen place epflen place',
 'thats standing ovation epflen',
 'time open farms pitch openfood opendata epfl',
 'good opportunity ethen epflen epflpark uzhnewsen',
 'exciting news world university rankings subject computer science eth epfl congrats httpstcoarslxzoshq',
 'epfl cryptotokenize free energy httpstcozyenraseir get rich',
 'scientists discover hidden patterns brain activity httpstcohblrytdmtt epfl',
 'edacyhq edacy invest',
 'deadline breast cancer research funding epfl againstbc',
 'robots transform furniture epfl november kamiahok',
 'microsoft president bradsmi epfl ebugnion digitalgenevaconvention digitaltrust important topics ethrat epflen',
 'atelier travail future traversée rade geneve cantongeneve epfl hslu etc hesso hei hepia heiafr',
 'formidable fan projet time machine donc possible revoir conférence chaîne youtube si jai bien lu les précédents commentaires merci signaler quand cette dernière ligne archives numerique',
 'members wef global university lead

In [18]:
term_doc_matrix_freq = construct_term_document_matrix(vocab_freq, documents)

IndexError: index 1001 is out of bounds for axis 0 with size 1001

## Perform LSI by selecting the first 100 largest singular values of the term document matrix  
Hint 1: np.linalg.svd(M, full_matrices=False) performs SVD on the matrix $\mathbf{M}$ and returns $\mathbf{K}, \mathbf{S}, \mathbf{D}^T$

 -  $\mathbf{K}, \mathbf{D}^T$ are matrices with orthonormal columns
 -  $\mathbf{S}$ is a **vector** of singular values in a **descending** order
 
Hint 2: np.diag(V) converts a vector to a diagonal matrix

Hint 3: To select 
 - The first k rows of a matrix A, use A[0:k, :]
 - The first k columns of a matrix A, use A[:, 0:k]
 - The submatrix from first k rows and k columns of a matrix A, use A[0:k, 0:k]

In [ ]:
# Input: a term document matrix and the number of singular values that will be selected
# Output: K_s, S_s, Dt_s are similar to the defintion in the lecture
def truncated_svd(term_doc_matrix, num_val):
    
    # YOUR CODE HERE
    K, S, Dt = np.linalg.svd(term_doc_matrix, full_matrices=False)
    K_s = K[:,0:num_val]
    S_s = np.diag(S)[0:num_val,0:num_val]
    Dt_s = Dt[0:num_val,:]
    
    return K_s, S_s, Dt_s


In [ ]:
K_s, S_s, Dt_s = truncated_svd(term_doc_matrix_freq, 100)

## Transform the given query 

In this question, you need to construct a vector representation for the following query:

In [ ]:
query = ['epfl']

### Treat the query as another document 

Hint: vocabulary.index(word) returns the index of the word if the word is in the vocabulary

In [ ]:
def query_to_document_vector(query, vocabulary):
    vector = np.zeros(len(vocabulary))
    for word in query:
        try:
            word_id = vocabulary.index(word)
            vector[word_id] = 1
            # YOUR CODE HERE
        except:
            # if query word is not in vocabulary, ignore it
            pass
    return vector


In [20]:
vector = np.zeros(5)
vector

array([0., 0., 0., 0., 0.])

### Compute the query vector from its document vector

Hint: 
 -  To compute inverse of a matrix M, use np.linalg.inv(M)
 -  To compute the dot product of A, B, use np.dot(A,B)

In [ ]:
def construct_query_vector(query, vocabulary, K_s, S_s, Dt_s):
    q = query_to_document_vector(query, vocabulary)
    
    S_sel_inv = np.linalg.inv(S_s)
    q_trans = np.dot(np.dot(q,K_s),S_sel_inv)
    # YOUR CODE HERE
    
    return q_trans


In [ ]:
query_vector_freq = construct_query_vector(query, vocab_freq, K_s, S_s, Dt_s)

## Retrieve top-10 relevant documents
In this question, you need to retrieve the top-10 documents that are relevant to the query using cosine similarity. You are given a function to compute the cosine simimlarity and a function that return the top-k documents given the query and document vectors.

In [ ]:
import math

def cosine_similarity(v1, v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxy == 0:
            sim = 0
    else:
            sim = sumxy/math.sqrt(sumxx*sumyy)
    return sim
    # YOUR CODE HERE
    
    
'''
Dt_sel_sim = np.transpose(Dt_sel)
sim_index = np.empty([4,2])
sim = []
for i in range(4):  
    sim_ = cosine_similarity(q_Star,Dt_sel[:,i])
    sim.append(sim_)
'''

In [ ]:
def retrieve_documents(query_vector, top_k, doc_vecs):
    scores = [[cosine_similarity(query_vector, doc_vecs[:,d]), d] for d in range(len(documents))]
    scores.sort(key=lambda x: -x[0])
    doc_ids = []
    retrieved = []
    for i in range(top_k):
        doc_ids.append(scores[i][1])
        retrieved.append(orig_docs[scores[i][1]])
    return doc_ids, retrieved

### Fill in the correct parameters in the following function

In [ ]:
retrieved_ids_freq, retrieved_docs_freq = retrieve_documents(# YOUR PARAMETERS HERE)

# Evaluation
In this question, we consider the scikit reference code as an “oracle” that supposedly gives the correct result. You need to compare your retrieval results with this oracle using the F1-score.

In [ ]:
# Retrieval oracle 
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), vocabulary=vocab_freq, min_df = 1, stop_words = 'english')
features = tf.fit_transform(documents)
npm_tfidf = features.todense()

# Return all document ids that that have cosine similarity with the query larger than a threshold
def search_vec_sklearn(query, features, threshold=0.3):
    new_features = tf.transform([query])
    cosine_similarities = linear_kernel(new_features, features).flatten()
    related_docs_indices, cos_sim_sorted = zip(*sorted(enumerate(cosine_similarities), key=itemgetter(1), 
                                                       reverse=True))
    doc_ids = []
    for i, cos_sim in enumerate(cos_sim_sorted):
        if cos_sim < threshold:
            break
        doc_ids.append(related_docs_indices[i])
    return doc_ids

In [ ]:
# gt_ids are the document ids retrieved by the oracle
gt_ids = search_vec_sklearn(" ".join(query), features)

## Compute F1-score at 10 between the oracle and your result

In [ ]:
def compute_recall_at_k(predicted, oracle, k):
    
    correct_recall=set(predicted[:k]).intersection(set(oracle[:k]))
    recall = len(correct_recall)/len(oracle)
    
    # YOUR CODE HERE
    
    return recall

In [ ]:
def compute_precision_at_k(predicted, oracle, k):
    
    correct_precision = set(predicted[:k]).intersection(set(oracle[:k]))
    precision = len(correct_precision)/k
    # YOUR CODE HERE
    
    return precision

In [ ]:
def f1score(predicted, oracle, k):
    P = compute_precision_at_k(predicted, oracle, k)
    R = compute_recall_at_k(predicted, oracle, k)
    up_score = 2*P*R
    low_score = P+R
    f1 = up_score/low_score
    # YOUR CODE HERE
    
    return f1

In [ ]:
f1score(retrieved_ids_freq, gt_ids, 10)

# Visualization

## Compute the term vectors using two principal concepts
Hint: you can reuse a method from the above

In [ ]:
# term_vecs_freq is a matrix of size (num_terms, 2)

vocab_freq = create_vocabulary_frequency(documents, 1000)
term_vecs_freq = construct_term_document_matrix(vocab_freq, documents)# YOUR CODE HERE 
K_ss, S_ss, Dt_ss = truncated_svd(term_doc_matrix_freq, 2)

query = ['epfl']
query_vector_freq = construct_query_vector(query, vocab_freq, K_ss, S_ss, Dt_ss)

retrieved_ids_freq, retrieved_docs_freq = retrieve_documents






'''
def create_vocabulary_frequency(corpus, vocab_len):
    
    
    count = {} # dictionary that contains the frequency of each word count[word] = freq
    for document in corpus:
        for word in document.split():
            if word not in count.keys():
                count[word] = 1
            else:
                count[word] += 1

    sorted_count_by_freq = sorted(count.items(), key=lambda kv: kv[1], reverse=True)
    vocabulary = [x[0] for x in sorted_count_by_freq[:vocab_len+1]]
    
    return vocabulary

def construct_term_document_matrix(vocabulary, documents):
    matrix = np.zeros((len(vocabulary), len(documents)))
    for j, document in enumerate(documents):
        counter = Counter(document.split())
        for i, word in enumerate(vocabulary):
            if word not in counter.keys():
                matrix[j][i] = 0
            else:
                matrix[j][i] = counter[word]
            # YOUR CODE HERE
            
    return matrix
    
'''

## Explain the scatter plot of the term vectors

In [ ]:
plt.scatter(term_vecs_freq[:, 0], term_vecs_freq[:, 1])
for i, t in enumerate(vocab_freq):
    plt.annotate(t, (term_vecs_freq[i, 0], term_vecs_freq[i, 1]))
plt.show()

Hint: Do you see any outliers? What is a possible explanation?

No. Becasue Top two sigular values are representive enough. Others are small values and noises. Not so useful and won't change the final result.